In [9]:
class TreeNode:
    '''
    Class for holding data of each node. Contains all information related to the person in the node.
    '''        
        
    def __init__(self,name,Tree):
        '''
        Create variables for the node
        '''
        self.name = name
        self.sex = None
        self.mother = None
        self.father = None
        self.children_ref = []  
        self.tree = Tree
        
    def sanitize_input(val):
        if(val.strip() == ""):
            return None
        elif(val.lower()=="y" or val.lower()=="yes"):
            return True
        elif(val.lower()=="n" or val.lower()=="no"):
            return False
        elif(val.lower()=="m" or val.lower()=="f"):
            return val.upper()
        else:
            return val

    def update(self):
        if(self.sex == None):
            self.sex = sanitize_input(input("Enter {}'s sex (m/f)".format(self.name)))
        if(self.mother == None):    
            self.mother = sanitize_input(input("Enter {}'s mother's name".format(self.name)))
        if(self.father == None):
            self.father = sanitize_input(input("Enter {}'s father's name".format(self.name)))
        
        i  = 0
        while(sanitize_input(input("{}'s {}th child? (y/n)".format(self.name,i+1)))):
            child = sanitize_input(input("Enter {}'s {}'th child's name".format(self.name,i+1)))
            self.tree.append(TreeNode(child))
            self.children_ref.append(i)
            i+=1
        
        if(self.mother!= None):
            self.tree.append(TreeNode(self.mother,self.tree))
            self.tree[-1].

SyntaxError: invalid syntax (<ipython-input-9-c8509e773f23>, line 17)

In [ ]:
#List to hold all the members
Full_tree = []